In [69]:
#Tensorflow Tutorial 참조
#https://www.tensorflow.org/tutorials/structured_data/preprocessing_layers?hl=ko

In [70]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [71]:
import sys
assert sys.version_info >= (3, 5)


import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [72]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler 
import csv
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [73]:
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "classification"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [74]:
import pandas as pd
import os
import tarfile
import urllib.request

ts=pd.read_csv("Features_testset.csv")
tv=pd.read_csv("Features_trainvalidation.csv")

In [75]:
tv['Bandgap'] = np.where(tv['Bandgap']== 0,0,1)

In [76]:
tv['Bandgap'].value_counts()

0    35906
1     8268
Name: Bandgap, dtype: int64

In [77]:
tv.head()

,NComp,Comp_L2Norm,Comp_L3Norm,Comp_L5Norm,Comp_L7Norm,Comp_L10Norm,mean_Number,maxdiff_Number,dev_Number,max_Number,...,frac_sValence,frac_pValence,frac_dValence,frac_fValence,CanFormIonic,MaxIonicChar,MeanIonicChar,Index,TotalEnergy,Bandgap
0,1,1.000000,1.000000,1.000000,1.000000,1.000000,89.000000,0,0.000000,89,...,0.666667,0.0,0.333333,0.000000,0,0.000000,0.00000,1,-4.4803,0
1,1,1.000000,1.000000,1.000000,1.000000,1.000000,47.000000,0,0.000000,47,...,0.090909,0.0,0.909091,0.000000,0,0.000000,0.00000,5,-3.2681,0
2,2,0.745356,0.693361,0.670782,0.667408,0.666732,57.666667,32,14.222222,79,...,0.063830,0.0,0.638298,0.297872,0,0.088829,0.03948,9,-3.5294,0
3,2,0.745356,0.693361,0.670782,0.667408,0.666732,57.666667,32,14.222222,79,...,0.063830,0.0,0.638298,0.297872,0,0.088829,0.03948,13,-3.5643,0
4,2,0.745356,0.693361,0.670782,0.667408,0.666732,57.666667,32,14.222222,79,...,0.063830,0.0,0.638298,0.297872,0,0.088829,0.03948,17,-3.4825,0


In [78]:
#상위 10개 상관관계 확인
corr_matrix = tv.corr()

df_corr_matrix = corr_matrix["Bandgap"].sort_values(axis=0, ascending=False)
df_corr_matrix.head(11)

Bandgap                  1.000000
CanFormIonic             0.691193
mean_NpValence           0.640377
frac_pValence            0.639319
most_NpValence           0.614357
max_NpValence            0.583238
max_Electronegativity    0.576072
maxdiff_NpValence        0.559717
MaxIonicChar             0.553415
MeanIonicChar            0.552481
dev_NpValence            0.541735
Name: Bandgap, dtype: float64

In [79]:
# 상관관계가 -0.2 미만 0.2 초과인 피쳐 확인

corr_matrix_2 = pd.DataFrame(corr_matrix.iloc[-1,:]).T

throw_away_col = []
for col in corr_matrix_2:
    if abs(-0.2 <= corr_matrix_2[col][0]) <= 0.2:
        throw_away_col.append(col)
print(throw_away_col)

['mean_Number', 'min_Number', 'most_Number', 'mean_AtomicWeight', 'min_AtomicWeight', 'most_AtomicWeight', 'mean_MeltingT', 'min_MeltingT', 'most_MeltingT', 'mean_Row', 'min_Row', 'most_Row', 'mean_CovalentRadius', 'min_CovalentRadius', 'most_CovalentRadius', 'mean_NdValence', 'most_NdValence', 'mean_NfValence', 'most_NfValence', 'mean_NdUnfilled', 'most_NdUnfilled', 'mean_NUnfilled', 'most_NUnfilled', 'mean_SpaceGroupNumber', 'min_SpaceGroupNumber', 'most_SpaceGroupNumber', 'frac_dValence', 'frac_fValence']


In [80]:
tv.shape

(44174, 148)

In [81]:
tv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44174 entries, 0 to 44173
Columns: 148 entries, NComp to Bandgap
dtypes: float64(96), int32(1), int64(51)
memory usage: 49.7 MB


In [82]:
tv.head()

,NComp,Comp_L2Norm,Comp_L3Norm,Comp_L5Norm,Comp_L7Norm,Comp_L10Norm,mean_Number,maxdiff_Number,dev_Number,max_Number,...,frac_sValence,frac_pValence,frac_dValence,frac_fValence,CanFormIonic,MaxIonicChar,MeanIonicChar,Index,TotalEnergy,Bandgap
0,1,1.000000,1.000000,1.000000,1.000000,1.000000,89.000000,0,0.000000,89,...,0.666667,0.0,0.333333,0.000000,0,0.000000,0.00000,1,-4.4803,0
1,1,1.000000,1.000000,1.000000,1.000000,1.000000,47.000000,0,0.000000,47,...,0.090909,0.0,0.909091,0.000000,0,0.000000,0.00000,5,-3.2681,0
2,2,0.745356,0.693361,0.670782,0.667408,0.666732,57.666667,32,14.222222,79,...,0.063830,0.0,0.638298,0.297872,0,0.088829,0.03948,9,-3.5294,0
3,2,0.745356,0.693361,0.670782,0.667408,0.666732,57.666667,32,14.222222,79,...,0.063830,0.0,0.638298,0.297872,0,0.088829,0.03948,13,-3.5643,0
4,2,0.745356,0.693361,0.670782,0.667408,0.666732,57.666667,32,14.222222,79,...,0.063830,0.0,0.638298,0.297872,0,0.088829,0.03948,17,-3.4825,0


In [83]:
df=pd.DataFrame(tv)
df.shape

(44174, 148)

# Keras Train

In [15]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(tv, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

28271 train examples
7068 validation examples
8835 test examples


In [16]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Bandgap')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

In [17]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)

In [18]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))

Every feature: ['NComp', 'Comp_L2Norm', 'Comp_L3Norm', 'Comp_L5Norm', 'Comp_L7Norm', 'Comp_L10Norm', 'mean_Number', 'maxdiff_Number', 'dev_Number', 'max_Number', 'min_Number', 'most_Number', 'mean_MendeleevNumber', 'maxdiff_MendeleevNumber', 'dev_MendeleevNumber', 'max_MendeleevNumber', 'min_MendeleevNumber', 'most_MendeleevNumber', 'mean_AtomicWeight', 'maxdiff_AtomicWeight', 'dev_AtomicWeight', 'max_AtomicWeight', 'min_AtomicWeight', 'most_AtomicWeight', 'mean_MeltingT', 'maxdiff_MeltingT', 'dev_MeltingT', 'max_MeltingT', 'min_MeltingT', 'most_MeltingT', 'mean_Column', 'maxdiff_Column', 'dev_Column', 'max_Column', 'min_Column', 'most_Column', 'mean_Row', 'maxdiff_Row', 'dev_Row', 'max_Row', 'min_Row', 'most_Row', 'mean_CovalentRadius', 'maxdiff_CovalentRadius', 'dev_CovalentRadius', 'max_CovalentRadius', 'min_CovalentRadius', 'most_CovalentRadius', 'mean_Electronegativity', 'maxdiff_Electronegativity', 'dev_Electronegativity', 'max_Electronegativity', 'min_Electronegativity', 'most_E

# 전처리

In [19]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for our feature.
  normalizer = preprocessing.Normalization(axis=None)

  # Prepare a Dataset that only yields our feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

In [20]:
photo_count_col = train_features['NComp']
layer = get_normalization_layer('NComp', train_ds)
layer(photo_count_col)

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.13652618, 0.13652618, 0.13652618, 0.13652618, 0.13652618],
      dtype=float32)>

In [21]:
#사용할 열 선택

batch_size = 256
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [22]:
all_inputs = []
encoded_features = []

# Numeric features.
for header in ['NComp', 'Comp_L2Norm', 'Comp_L3Norm', 'Comp_L5Norm', 'Comp_L7Norm', 'Comp_L10Norm', 'mean_Number', 'maxdiff_Number', 
               'dev_Number', 'max_Number', 'min_Number', 'most_Number', 'mean_MendeleevNumber', 'maxdiff_MendeleevNumber', 
               'dev_MendeleevNumber', 'max_MendeleevNumber', 'min_MendeleevNumber', 'most_MendeleevNumber', 'mean_AtomicWeight', 
               'maxdiff_AtomicWeight', 'dev_AtomicWeight', 'max_AtomicWeight', 'min_AtomicWeight', 'most_AtomicWeight', 
               'mean_MeltingT', 'maxdiff_MeltingT', 'dev_MeltingT', 'max_MeltingT', 'min_MeltingT', 'most_MeltingT', 'mean_Column', 
               'maxdiff_Column', 'dev_Column', 'max_Column', 'min_Column', 'most_Column', 'mean_Row', 'maxdiff_Row', 'dev_Row', 
               'max_Row', 'min_Row', 'most_Row', 'mean_CovalentRadius', 'maxdiff_CovalentRadius', 'dev_CovalentRadius', 
               'max_CovalentRadius', 'min_CovalentRadius', 'most_CovalentRadius', 'mean_Electronegativity', 
               'maxdiff_Electronegativity', 'dev_Electronegativity', 'max_Electronegativity', 'min_Electronegativity', 
               'most_Electronegativity', 'mean_NsValence', 'maxdiff_NsValence', 'dev_NsValence', 'max_NsValence', 
               'min_NsValence', 'most_NsValence', 'mean_NpValence', 'maxdiff_NpValence', 'dev_NpValence', 'max_NpValence', 
               'min_NpValence', 'most_NpValence', 'mean_NdValence', 'maxdiff_NdValence', 'dev_NdValence', 'max_NdValence', 
               'min_NdValence', 'most_NdValence', 'mean_NfValence', 'maxdiff_NfValence', 'dev_NfValence', 'max_NfValence', 
               'min_NfValence', 'most_NfValence', 'mean_NValance', 'maxdiff_NValance', 'dev_NValance', 'max_NValance', 
               'min_NValance', 'most_NValance', 'mean_NsUnfilled', 'maxdiff_NsUnfilled', 'dev_NsUnfilled', 'max_NsUnfilled',
               'min_NsUnfilled', 'most_NsUnfilled', 'mean_NpUnfilled', 'maxdiff_NpUnfilled', 'dev_NpUnfilled', 'max_NpUnfilled',
               'min_NpUnfilled', 'most_NpUnfilled', 'mean_NdUnfilled', 'maxdiff_NdUnfilled', 'dev_NdUnfilled', 'max_NdUnfilled', 
               'min_NdUnfilled', 'most_NdUnfilled', 'mean_NfUnfilled', 'maxdiff_NfUnfilled', 'dev_NfUnfilled', 'max_NfUnfilled', 
               'min_NfUnfilled', 'most_NfUnfilled', 'mean_NUnfilled', 'maxdiff_NUnfilled', 'dev_NUnfilled', 'max_NUnfilled', 
               'min_NUnfilled', 'most_NUnfilled', 'mean_GSvolume_pa', 'maxdiff_GSvolume_pa', 'dev_GSvolume_pa', 'max_GSvolume_pa', 
               'min_GSvolume_pa', 'most_GSvolume_pa', 'mean_GSbandgap', 'maxdiff_GSbandgap', 'dev_GSbandgap', 'max_GSbandgap', 
               'min_GSbandgap', 'most_GSbandgap', 'mean_GSmagmom', 'maxdiff_GSmagmom', 'dev_GSmagmom', 'max_GSmagmom', 
               'min_GSmagmom', 'most_GSmagmom', 'mean_SpaceGroupNumber', 'maxdiff_SpaceGroupNumber', 'dev_SpaceGroupNumber', 
               'max_SpaceGroupNumber', 'min_SpaceGroupNumber', 'most_SpaceGroupNumber', 'frac_sValence', 'frac_pValence', 
               'frac_dValence', 'frac_fValence', 'CanFormIonic', 'MaxIonicChar', 'MeanIonicChar']:
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_ds)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)

In [23]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(all_inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

# Keras

In [24]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10


C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\functional.py:559: UserWarning: Input dict contained keys ['Index', 'TotalEnergy'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


111/111 [==============================] - 5s 23ms/step - loss: 0.3948 - accuracy: 0.8379 - val_loss: 0.2323 - val_accuracy: 0.8977
Epoch 2/10
111/111 [==============================] - 3s 16ms/step - loss: 0.2339 - accuracy: 0.8988 - val_loss: 0.2130 - val_accuracy: 0.9012
Epoch 3/10
111/111 [==============================] - 3s 16ms/step - loss: 0.2151 - accuracy: 0.9065 - val_loss: 0.2036 - val_accuracy: 0.9131
Epoch 4/10
111/111 [==============================] - 3s 16ms/step - loss: 0.2036 - accuracy: 0.9136 - val_loss: 0.1990 - val_accuracy: 0.9144
Epoch 5/10
111/111 [==============================] - 3s 16ms/step - loss: 0.1986 - accuracy: 0.9160 - val_loss: 0.1943 - val_accuracy: 0.9182
Epoch 6/10
111/111 [==============================] - 3s 16ms/step - loss: 0.1938 - accuracy: 0.9186 - val_loss: 0.1908 - val_accuracy: 0.9211
Epoch 7/10
111/111 [==============================] - 3s 15ms/step - loss: 0.1896 - accuracy: 0.9173 - val_loss: 0.1902 - val_accuracy: 0.9215
Epoch 8/10

In [25]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

35/35 [==============================] - 1s 13ms/step - loss: 0.1722 - accuracy: 0.9288
Accuracy 0.9288058876991272


# 모델 검증

In [32]:
ts.drop(['Index'], axis = 1, inplace = True)